In [ ]:
!pip install accelerate -Uq

exit()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.3 MB/s eta 0:00:00


In [ ]:
!pip install -q datasets transformers evaluate

# Сделаем torch Dataset

In [ ]:
from transformers import set_seed
import os
import random
import numpy as np

import torch

def set_all_seeds(seed=22):

    # python's seeds
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    # torch's seeds
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


magic_seed = 22
set_all_seeds(seed=magic_seed)

# 6 дней после заражения

# Сделаем torch Dataset

In [ ]:
# Import libraries
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os
import numpy as np
import pandas as pd
import torch

# Mount google drive
from google.colab import drive
drive.mount('gdrive')

## Подготовка набора данных

In [ ]:
class MultispecDataset(Dataset):
    # create dataset of multispectral images
    def __init__(self, nir_dir, rgb_dir, mask_dir, img_names, transform=None):
        '''
        nir_dir:
            path to NIR images

        rgb_dir:
            path to RGB images

        mask_dir:
            path to masks

        image_names:
            names of images
        '''
        self.nir_dir = nir_dir
        self.rgb_dir = rgb_dir
        self.mask_dir = mask_dir
        self.img_names = img_names
        self.transform = transform

    def __getitem__(self, idx):
        # Open image
        nir_img = Image.open(os.path.join(self.nir_dir, self.img_names[idx]))

        # transform to tensor, change size from [512, 512, 3] to [3, 512, 512]
        # and scale pixel values to range [0, 1]
        nir_img = torch.tensor(np.array(nir_img)).permute((2, 0, 1)) / 255

        rgb_img = Image.open(os.path.join(self.rgb_dir, self.img_names[idx]))
        rgb_img = torch.tensor(np.array(rgb_img)).permute((2, 0, 1)) / 255

        mask = Image.open(os.path.join(self.mask_dir, self.img_names[idx].replace('JPG', 'png')))

        # transform if desired
        if self.transform:
            nir_img = self.transform(nir_img)

        # concatenate images to get the multispectral one
        image = torch.cat([nir_img, rgb_img], axis=0)

        return {'pixel_values': image, 'labels': torch.tensor(np.array(mask) / 255).long()}

    def __len__(self):
        return len(self.img_names)

In [ ]:
id2label = {0: 'unlabeled', 1: 'leaf-disease'}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)

In [ ]:
id2label

{0: 'unlabeled', 1: 'leaf-disease'}

In [ ]:
from transformers import SegformerImageProcessor

model_checkpoint =  "nvidia/mit-b2"
image_processor = SegformerImageProcessor.from_pretrained(model_checkpoint, do_reduce_labels=False)

In [ ]:
image_mean = image_processor.image_mean
image_std = image_processor.image_std
image_size = image_processor.size

print('image_mean:', image_mean)
print('image_std:', image_std)
print('image_size:',image_size)

image_mean: [0.485, 0.456, 0.406]
image_std: [0.229, 0.224, 0.225]
image_size: {'height': 512, 'width': 512}


In [ ]:
image_transforms = transforms.Compose([
                                       transforms.Resize(size=(image_size['height'], image_size['width'])),
                                       transforms.Normalize(mean=image_mean,std=image_std)])

In [ ]:
# Image directories
nir_dir = '/content/gdrive/MyDrive/Cropped Potato/2023_ИК_больные/6 дней после заражения'
rgb_dir = '/content/gdrive/MyDrive/Cropped Potato/2023_ВИД_больные/6 дней после заражения'
mask_dir = '/content/gdrive/MyDrive/Cropped Potato/Разметка/6 дней после заражения/Разметка картинки'

In [ ]:
rgb_images = os.listdir(rgb_dir)
mask_images = os.listdir(mask_dir)

In [ ]:
# 0.7 : 0.15 : 0.15
num_train_samples = 945
num_validation_samples = 210
num_test_samples = len(rgb_images) - num_train_samples - num_validation_samples # 189

In [ ]:
# Make train-val-test split
train_images = rgb_images[:num_train_samples]
train_masks = mask_images[:num_train_samples]

val_images = rgb_images[num_train_samples:(num_train_samples + num_validation_samples)]
val_masks = mask_images[num_train_samples:(num_train_samples + num_validation_samples)]

test_images = rgb_images[(num_train_samples + num_validation_samples):]
test_masks = mask_images[(num_train_samples + num_validation_samples):]

In [ ]:
print(len(train_images), len(train_masks))
print(len(val_images), len(val_masks))
print(len(test_images), len(test_masks))

945 945
210 210
189 189


In [ ]:
# Create Dataset objects
train_dataset = MultispecDataset(nir_dir, rgb_dir, mask_dir, train_images, image_transforms)
val_dataset = MultispecDataset(nir_dir, rgb_dir, mask_dir, val_images, image_transforms)

## Tune HP

### Write custom Trainers

We are going to write custom trainers (with custom loss function)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch import nn
from transformers import Trainer

In [ ]:
!pip install segmentation-models-pytorch -q

In [ ]:
# Weighted Cross-Entropy
class WeightedCSETrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits_tensor = nn.functional.interpolate(
        logits,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
        )

        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([self.weight_0, self.weight_1]).to(device))
        loss = loss_fct(logits_tensor.reshape(-1, self.model.config.num_labels), labels.reshape(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
from segmentation_models_pytorch.losses import JaccardLoss

# Jaccard Loss
class JaccardTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits_tensor = nn.functional.interpolate(
        logits,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
        )

        disease_logits = logits_tensor[:, 1, :, :]


        loss_fct = JaccardLoss(mode='binary')
        loss = loss_fct(disease_logits,
                        labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
from segmentation_models_pytorch.losses import DiceLoss

# Dice Loss
class DiceTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits_tensor = nn.functional.interpolate(
        logits,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
        )

        disease_logits = logits_tensor[:, 1, :, :]


        loss_fct = DiceLoss(mode='binary')
        loss = loss_fct(disease_logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
from segmentation_models_pytorch.losses import TverskyLoss

# Tversky Loss
class TverskyTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits_tensor = nn.functional.interpolate(
        logits,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
        )

        disease_logits = logits_tensor[:, 1, :, :]


        loss_fct = TverskyLoss(mode='binary',
                               alpha=self.alpha, beta=self.betta)
        loss = loss_fct(disease_logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
from segmentation_models_pytorch.losses import FocalLoss

# Focal Loss
class FocalTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits_tensor = nn.functional.interpolate(
        logits,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
        )

        disease_logits = logits_tensor[:, 1, :, :]


        loss_fct = FocalLoss(mode='binary',
                               alpha=self.alpha, gamma=self.gamma)
        loss = loss_fct(disease_logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
from segmentation_models_pytorch.losses import LovaszLoss

# Lovasz Loss
class LovaszTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits_tensor = nn.functional.interpolate(
        logits,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
        )

        disease_logits = logits_tensor[:, 1, :, :]


        loss_fct = LovaszLoss(mode='binary')
        loss = loss_fct(disease_logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
from segmentation_models_pytorch.losses import MCCLoss

# MCC Loss
class MCCTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        logits_tensor = nn.functional.interpolate(
        logits,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
        )

        disease_logits = logits_tensor[:, 1, :, :]


        loss_fct = MCCLoss()
        loss = loss_fct(disease_logits, labels)

        return (loss, outputs) if return_outputs else loss

### Wandb

In [ ]:
!pip install -q wandb

In [ ]:
import wandb
wandb.login()

%env WANDB_PROJECT=leaf_disease_sweeps
%env WANDB_LOG_MODEL='end'

In [ ]:
model_checkpoint = "nvidia/mit-b2"

In [ ]:
from transformers import SegformerForSemanticSegmentation, TrainingArguments, Trainer
import torch.nn as nn

def model_init():
    '''
    initialize model

    Return:
    -------
        model to fine-tune
    '''
    model = SegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,  # Will ensure the segmentation specific components are reinitialized.
    )

    # Change model architecture so that it will be able to work with multispectral (6-channel) images
    model.segformer.encoder.patch_embeddings[0].proj = nn.Conv2d(6, 64,
                                                             kernel_size=(7, 7),
                                                             stride=(4, 4),
                                                             padding=(3, 3))
    return model

In [ ]:
# Define sweep config
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'eval/mean',
        'goal': 'maximize'
    }
}

# hyperparameters
parameters_dict = {
    'epochs': {
        'value': 5
        },

    'batch_size': {
        'value': 8
    },

    'num_warmup_steps': {
        'distribution': 'int_uniform',
        'min': 0,
        'max': 200,
    },

    'learning_rate': {
        'distribution': 'uniform',
        'min': 1e-6,
        'max': 1e-3
    },

    'weight_decay': {
        'distribution': 'uniform',
        'min': 0.0,
        'max': 1.5,
    },

    'objective': {
        'values': ['CrossEntropyLoss', 'JaccardLoss', 'DiceLoss', 'TverskyLoss',
                   'FocalLoss', 'LovaszLoss', 'MCCLoss']
    },

    'betta_1': {
        'distribution': 'uniform',
        'min': 0.5,
        'max': 1
    },

    'betta_2': {
        'distribution': 'uniform',
        'min': 0.5,
        'max': 1
    },

    'class_weight_0': {
        'distribution': 'uniform',
        'min': 0.3,
        'max': 5
    },

    'class_weight_1': {
        'distribution': 'uniform',
        'min': 5,
        'max': 50
    },

    'tversky_alpha': {
        'distribution': 'uniform',
        'min': 0.1,
        'max': 1
    },

    'tversky_betta': {
        'distribution': 'uniform',
        'min': 0.1,
        'max': 1
    },

    'focal_gamma': {
        'distribution': 'uniform',
        'min': 0,
        'max': 5
    },

    'focal_alpha': {
        'distribution': 'uniform',
        'min': 0.1,
        'max': 1
    },

    'smooth': {
        'distribution': 'uniform',
        'min': 0,
        'max': 1.5
    },
}


sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project='multispec-leaf-disease-sweeps')

In [ ]:
from sklearn.metrics import cohen_kappa_score

def calculate_cohen_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true.flatten(), y_pred.flatten())

In [ ]:
import torch
from torch import nn
import evaluate
from matplotlib import pyplot as plt

metric = evaluate.load("mean_iou")
sigma = nn.Sigmoid()

def compute_metrics(eval_pred):
    global best_mean
    global best_iou
    global best_dice
    global best_kappa
    with torch.no_grad():

        logits, labels = eval_pred

        logits_tensor = torch.from_numpy(logits)
        # scale the logits to the size of the label
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).detach().cpu().numpy()


        pred_labels = logits_tensor.argmax(axis=1)

        # currently using _compute instead of compute
        # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
        metrics = metric._compute(
                predictions=pred_labels,
                references=labels,
                num_labels=len(id2label),
                ignore_index=-1,
                reduce_labels=False,#feature_extractor.do_reduce_labels,
            )


        eval_iou = metrics['per_category_iou'][1]

        metrics = {'eval_IoU': eval_iou}
        metrics['eval_Dice'] = eval_iou * 2 / (eval_iou + 1)
        metrics['eval_kappa_score'] = calculate_cohen_kappa(labels, pred_labels)
        metrics['eval_mean'] = (metrics['eval_IoU'] + metrics['eval_kappa_score']) / 2

        print('Mean:', metrics['eval_mean'])
        print('IoU:', metrics['eval_IoU'])
        print('Dice:', metrics['eval_Dice'])
        print('kappa:', metrics['eval_kappa_score'])

        if metrics['eval_mean'] > best_mean:
            best_mean = metrics['eval_mean']
            best_iou = metrics['eval_IoU']
            best_dice = metrics['eval_Dice']
            best_kappa = metrics['eval_kappa_score']
        else:
            metrics['eval_mean'] = best_mean
            metrics['eval_IoU'] = best_iou
            metrics['eval_Dice'] = best_dice
            metrics['eval_kappa_score'] = best_kappa


    return metrics

In [ ]:
from torch.optim import AdamW
from transformers import TrainingArguments, get_linear_schedule_with_warmup

def train(config=None):
    '''
    perform training procedure with particular configuration

    Parameters:
    -----------
        config (dict):
            training hyperparameters
    '''

    global best_mean
    global best_iou
    global best_dice
    global best_kappa
    best_mean = 0
    best_iou = 0
    best_dice = 0
    best_kappa = 0
    with wandb.init(config=config):
        # set sweep configuration
        config = wandb.config

        epochs = config.epochs
        batch_size = config.batch_size

        num_training_steps = len(train_dataset) // batch_size * epochs

        model = model_init()
        optimizer = AdamW(model.parameters(), lr=config.learning_rate,
                          weight_decay=config.weight_decay,
                          betas=(config.betta_1, config.betta_1))
        scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                                    num_warmup_steps=config.num_warmup_steps,
                                                    num_training_steps=num_training_steps)



        # set training arguments
        training_args = TrainingArguments(
        "segformer-b2-finetuned-leaf-disease",
        label_smoothing_factor=config.smooth,
        num_train_epochs=config.epochs,
        per_device_train_batch_size=config.batch_size,
        per_device_eval_batch_size=config.batch_size,
        save_total_limit=5,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='eval_mean',
        greater_is_better=True,
        push_to_hub=False,
        remove_unused_columns=False,
        seed=magic_seed,
        hub_strategy="end",
        )

        objective = config.objective

        # define particular Trainer
        if objective == 'CrossEntropyLoss':
            trainer = WeightedCSETrainer(model=model,
                                         args=training_args,
                                         train_dataset=train_dataset,
                                         eval_dataset=val_dataset,
                                         compute_metrics=compute_metrics,
                                         optimizers=(optimizer, scheduler))
            setattr(trainer, 'weight_0', config.class_weight_0)
            setattr(trainer, 'weight_1', config.class_weight_1)
        elif objective == 'JaccardLoss':
            trainer = JaccardTrainer(model=model,
                                     args=training_args,
                                     train_dataset=train_dataset,
                                     eval_dataset=val_dataset,
                                     compute_metrics=compute_metrics,
                                     optimizers=(optimizer, scheduler))

        elif objective == 'DiceLoss':
            trainer = DiceTrainer(model=model,
                                  args=training_args,
                                  train_dataset=train_dataset,
                                  eval_dataset=val_dataset,
                                  compute_metrics=compute_metrics,
                                  optimizers=(optimizer, scheduler))

        elif objective == 'TverskyLoss':
            trainer = TverskyTrainer(model=model,
                                     args=training_args,
                                     train_dataset=train_dataset,
                                     eval_dataset=val_dataset,
                                     compute_metrics=compute_metrics,
                                     optimizers=(optimizer, scheduler))

            setattr(trainer, 'alpha', config.tversky_alpha)
            setattr(trainer, 'betta', config.tversky_betta)
        elif objective == 'FocalLoss':
            trainer = FocalTrainer(model=model,
                                   args=training_args,
                                   train_dataset=train_dataset,
                                   eval_dataset=val_dataset,
                                   compute_metrics=compute_metrics,
                                   optimizers=(optimizer, scheduler))

            setattr(trainer, 'alpha', config.focal_alpha)
            setattr(trainer, 'gamma', config.focal_gamma)
        elif objective == 'LovaszLoss':
            trainer = LovaszTrainer(model=model,
                                    args=training_args,
                                    train_dataset=train_dataset,
                                    eval_dataset=val_dataset,
                                    compute_metrics=compute_metrics,
                                    optimizers=(optimizer, scheduler))
        elif objective == 'MCCLoss':
            trainer = MCCTrainer(model=model,
                                 args=training_args,
                                 train_dataset=train_dataset,
                                 eval_dataset=val_dataset,
                                 compute_metrics=compute_metrics,
                                 optimizers=(optimizer, scheduler))
        else:
            print('ВЫ ВВЕЛИ НЕПРАВИЛЬНЫЙ ЛОСС')



        # start training loop
        trainer.train()

In [ ]:
# We will log the mean value of IoU and kappa score. The higher this value, the better
best_mean = 0

best_iou = 0
best_dice = 0
best_kappa = 0

In [ ]:
# Start tuning hyperparameters
wandb.agent(sweep_id, train, count=50)

wandb: Agent Starting Run: 20ji1jbq with config:
wandb: 	batch_size: 8
wandb: 	betta_1: 0.6733811349468842
wandb: 	betta_2: 0.6531000450935891
wandb: 	class_weight_0: 2.5335889266254115
wandb: 	class_weight_1: 29.45220495433291
wandb: 	epochs: 5
wandb: 	focal_alpha: 0.1261787953389969
wandb: 	focal_gamma: 1.36537336321138
wandb: 	learning_rate: 0.00029416366340114924
wandb: 	num_warmup_steps: 91
wandb: 	objective: MCCLoss
wandb: 	smooth: 0.7543296344281722
wandb: 	tversky_alpha: 0.1841302902763169
wandb: 	tversky_betta: 0.16571841329169207
wandb: 	weight_decay: 1.3090100498640664


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b2 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.wei

Epoch,Training Loss,Validation Loss,Iou,Dice,Kappa Score,Mean
1,0.781100,nan,0,0,0,0
2,0.000000,nan,0,0,0,0
3,0.000000,nan,0,0,0,0
4,0.000000,nan,0,0,0,0
5,0.000000,nan,0,0,0,0


Mean: 0.0
IoU: 0.0
Dice: 0.0
kappa: 0.0
Mean: 0.0
IoU: 0.0
Dice: 0.0
kappa: 0.0
Mean: 0.0
IoU: 0.0
Dice: 0.0
kappa: 0.0
Mean: 0.0
IoU: 0.0
Dice: 0.0
kappa: 0.0
Mean: 0.0
IoU: 0.0
Dice: 0.0
kappa: 0.0


eval/Dice,▁▁▁▁▁
eval/IoU,▁▁▁▁▁
eval/kappa_score,▁▁▁▁▁
eval/mean,▁▁▁▁▁
eval/runtime,█▆▅▂▁
eval/samples_per_second,▁▃▄▇█
eval/steps_per_second,▁▃▄██
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███
train/learning_rate,█▆▄▃▁
train/loss,█▁▁▁▁


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 1auic82e with config:
wandb: 	batch_size: 8
wandb: 	betta_1: 0.9552723961619992
wandb: 	betta_2: 0.8690295460589099
wandb: 	class_weight_0: 1.790318702394839
wandb: 	class_weight_1: 18.14968159440661
wandb: 	epochs: 5
wandb: 	focal_alpha: 0.5579754055573171
wandb: 	focal_gamma: 3.726141630312916
wandb: 	learning_rate: 0.00024195991091649285
wandb: 	num_warmup_steps: 97
wandb: 	objective: LovaszLoss
wandb: 	smooth: 0.2321194661159769
wandb: 	tversky_alpha: 0.678223264777535
wandb: 	tversky_betta: 0.6158880382491824
wandb: 	weight_decay: 0.04521691935476141


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b2 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.wei

Epoch,Training Loss,Validation Loss,Iou,Dice,Kappa Score,Mean
1,1.520400,1.162397,0.004355,0.008671,-0.003567,0.000394
2,1.057900,1.008640,0.004355,0.008671,-0.003567,0.000394


Mean: 0.00039364822142389346
IoU: 0.004354506544332539
Dice: 0.008671254056130092
kappa: -0.0035672101014847524
Mean: -3.848738709955057e-06
IoU: 0.0
Dice: 0.0
kappa: -7.697477419910115e-06


wandb: Ctrl + C detected. Stopping sweep.
